## Zero-Shot

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.llms import OpenAI

model = OpenAI()
prompt = PromptTemplate(
    input_variables=["text"], template="Classify the sentiment of this text: {text}"
)
chain = prompt | model
chain.invoke({"text": "I hated that movie, it was terrible!"})

'\n\nNegative'

## Few-Shot

In [22]:
from langchain.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai.llms import OpenAI
from langchain_openai.embeddings import OpenAIEmbeddings


model = OpenAI()

example_prompt = PromptTemplate(
    template="{input} -> {output}",
    input_variables=["input", "output"],
)
examples = [
    {
        "input": "I absolutely love the new update! Everything works seamlessly.",
        "output": "Positive",
    },
    {
        "input": "It's okay, but I think it could use more features.",
        "output": "Neutral",
    },
    {
        "input": "I'm disappointed with the service, I expected much better performance.",
        "output": "Negative",
    },
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

chain = prompt | model
chain.invoke({"input": "This is an excellent book with high quality explanations."})

' -> Positive'

To choose examples tailored to each input, FewShotPromptTemplate can accept a SemanticSim  ilarityExampleSelector, based on embeddings rather than hardcoded examples. The Semant  icSimilarityExampleSelector automatically finds the most relevant examples for each input.  For many tasks, standard few-shot prompting works well, but there are many other techniques  and extensions when dealing with more complex reasoning tasks.

In [23]:
selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=Chroma,
    k=4,
)
prompt = FewShotPromptTemplate(
    example_selector=selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)
chain = prompt | model
chain.invoke({"input": "What's 10+10?"})

' -> Neutral'

## CoT

There are two variants of CoT:
- **zero-shot**
- **few-shot**


In zero-shot CoT, we just add the instruction “Let’s think step by step!” to the prompt.  

When asking an LLM to reason through a problem, it is often more effective to have it explain its  reasoning before stating the final answer. This encourages the LLM to logically think through  the problem first, rather than just guessing the answer and trying to justify it afterward. Asking  an LLM to explain its thought process aligns well with its core capabilities.

### Zero-Shot-CoT

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.llms import OpenAI

cot_instruction = "Let's think step by step!"
cot_instruction2 = "Explain your reasoning step-by-step. Finally, state the answer."

reasoning_prompt = "{question}\n" + cot_instruction

prompt = PromptTemplate(template=reasoning_prompt, input_variables=["question"])

model = OpenAI()
chain = prompt | model
chain.invoke(
        {
            "question": "There were 5 apples originally. I ate 2 apples. "
            "My friend gave me 3 apples. How many apples do I have now?",
        })

'\nStep 1: Start with 5 apples\nStep 2: Ate 2 apples\n5 - 2 = 3 apples\nStep 3: Friend gave 3 apples\n3 + 3 = 6 apples\nAnswer: You now have 6 apples.'

### Few-Shot-CoT

In [36]:
model = OpenAI()

example_prompt = PromptTemplate(
    template="{input} Reason: {output}",
    input_variables=["input", "output"],
)

examples = [
    {
        "input": "I absolutely love the new update! Everything works seamlessly.", 
        "output": "Love and absolute works seamlessly are examples of positive sentiment. Therefore, the sentiment is positive",
    },
    {  
        "input": "It's okay, but I think it could use more features.",
        "output": "It's okay is not an endorsement. The customer further thinks it should be extended. Therefore, the sentiment is neutral",
    }, 
    {
        "input": "I'm disappointed with the service, I expected much better performance.",
        "output": "The customer is disappointed and expected more. This is negative"
    }
]

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

chain = prompt | model
chain.invoke({"input": "This is an excellent book with high quality explanations."})

' Reason: The statement is a question and does not express a sentiment. Therefore, the sentiment is neutral.'

## Self-Consistency

With self-consistency prompting, the model generates multiple candidate answers to a question.  These are then compared against each other, and the most consistent or frequent answer is selected  as the final output. A good example of self-consistency prompting with LLMs is in the context of  fact verification or information synthesis, where accuracy is paramount.

In [1]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models.base import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

solutions_template = """
                    Generate {num_solutions} distinct answers to this question:
                    {question}

                    Solutions:
                    """

solutions_prompt = PromptTemplate(
    template=solutions_template,
    input_variables=["question", "num_solutions"]
)

llm = ChatOpenAI()
solutions_chain = solutions_prompt | llm | StrOutputParser()

consistency_template = """
                        For each answer in {solutions}, count the number of times it occurs. Finally,
                         choose the answer that occurs most.

                        Most frequent solution:
                        """

consistency_prompt = PromptTemplate(template=consistency_template, input_variables=["solutions"])
consistency_chain = consistency_prompt | llm | StrOutputParser()

answer_chain = solutions_chain | consistency_chain | StrOutputParser()

answer_chain.invoke(
    input={
        'question': "Which year was the Declaration of Independence of the United States signed?",
        'num_solutions': "5",
    }
)

'The answer "The Declaration of Independence of the United States was signed in 1776" occurs 5 times, making it the most frequent solution.'

## ToT

In Tree-of-Thought (ToT) prompting, we generate multiple problem-solving steps or approaches  for a given prompt and then use the AI model to critique them. The critique will be based on the  model’s judgment of the solution’s suitability to the problem.

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

solutions_template = """
Generate {num_solutions} distinct solutions for {problem}. Consider factors like {factors}.

Solutions:
"""
solutions_prompt = PromptTemplate(
    template=solutions_template, input_variables=["problem", "factors", "num_solutions"]
)

evaluation_template = """
Evaluate each solution in {solutions} by analyzing pros, cons, feasibility,
 and probability of success.

Evaluations:
"""
evaluation_prompt = PromptTemplate(template=evaluation_template, input_variables=["solutions"])

reasoning_template = """
For the most promising solutions in {evaluations}, explain scenarios, implementation strategies,
 partnerships needed, and handling potential obstacles. 

Enhanced Reasoning: 
"""
reasoning_prompt = PromptTemplate(template=reasoning_template, input_variables=["evaluations"])

ranking_template = """
Based on the evaluations and reasoning, rank the solutions in {enhanced_reasoning} from
 most to least promising.

Ranked Solutions:
"""
ranking_prompt = PromptTemplate(template=ranking_template, input_variables=["enhanced_reasoning"])

llm=ChatOpenAI()
output_parser = StrOutputParser()

solutions_chain = solutions_prompt| llm |output_parser
evaluation_chain = evaluation_prompt| llm |output_parser
reasoning_chain =  reasoning_prompt| llm |output_parser
ranking_chain =  ranking_prompt| llm |output_parser

tot_chain = solutions_chain | evaluation_chain | reasoning_chain | ranking_chain

tot_chain.invoke(
    input= {
        'problem': "Prompt engineering",
        'factors': "Requirements for high task performance, low token use, and few calls to the LLM",
        'num_solutions': 3,
        }
)

'1. Implementing a comprehensive training program for engineers by partnering with a professional training organization\n2. Utilizing advanced technology tools to automate tasks by partnering with a technology vendor\n3. Developing standardized engineering guidelines and best practices by forming a cross-functional team of experienced engineers'